In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import transformer_lens
import torch

/opt/conda/envs/mats_sae_training/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/opt/conda/envs/mats_sae_training/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/opt/conda/envs/mats_sae_training/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [3]:
from transformer_lens import HookedTransformer

In [5]:
# device =

# sample_input = torch.tensor([[
#     20, 19, 18, 10,  2,  1, 27,  3, 41, 42, 34, 12,  4, 40, 11, 29, 43, 13, 48, 56, 
#     33, 39, 22, 44, 24,  5, 46,  6, 32, 36, 51, 58, 52, 60, 21, 53, 26, 31, 37,  9,
#     25, 38, 23, 50, 45, 17, 47, 28, 35, 30, 54, 16, 59, 49, 57, 14, 15, 55, 7
# ]]).to(device)
    
# # The argmax of the output (ie the most likely next move from each position)
# sample_output = torch.tensor([[
#     21, 41, 40, 34, 40, 41,  3, 11, 21, 43, 40, 21, 28, 50, 33, 50, 33,  5, 33,  5,
#     52, 46, 14, 46, 14, 47, 38, 57, 36, 50, 38, 15, 28, 26, 28, 59, 50, 28, 14, 28, 
#     28, 28, 28, 45, 28, 35, 15, 14, 30, 59, 49, 59, 15, 15, 14, 15,  8,  7,  8
# ]]).to(device)

# assert (model(sample_input).argmax(dim=-1) == sample_output.to(device)).all()

In [23]:
dataset_path = 'taufeeque/othellogpt'
model_name = 'othello-gpt'

model = HookedTransformer.from_pretrained(model_name)

Loaded pretrained model othello-gpt into HookedTransformer


In [24]:
from sae_training.activations_store import ActivationsStore
from sae_training.config import LanguageModelSAERunnerConfig

In [27]:
config = LanguageModelSAERunnerConfig(
    model_name=model_name,
    hook_point="blocks.6.hook_resid_pre",
    hook_point_layer=6,
    dataset_path=dataset_path,
    context_size=model.cfg.n_ctx,
    d_in=model.cfg.d_model,
    n_batches_in_buffer=32,
    total_training_tokens=10*(1e6), # todo: more?
    store_batch_size=32,
    device=model.cfg.device,
    seed=42,
    dtype=torch.float32, # todo: float16
    b_dec_init_method="mean", # todo: geometric_median
    expansion_factor=4, # todo: adjust
    l1_coefficient=0.001, # todo: adjust
    lr=0.0003,
    lr_scheduler_name="constantwithwarmup",
    lr_warm_up_steps=5000,
    train_batch_size=4096,
    use_ghost_grads=True,
    feature_sampling_window=500,
    dead_feature_window=1e6,
    log_to_wandb=False,
    wandb_project="othello_gpt_sae",
    wandb_log_frequency=30,
    n_checkpoints=0,
    checkpoint_path="checkpoints"
)

Run name: 2048-L1-0.001-LR-0.0003-Tokens-1.000e+07
n_tokens_per_buffer (millions): 0.060416
Lower bound: n_contexts_per_buffer (millions): 0.001024
Total training steps: 2441.0
Total wandb updates: 81.0
n_tokens_per_feature_sampling_window (millions): 120.832
n_tokens_per_dead_feature_window (millions): 241664.0
Using Ghost Grads.
We will reset the sparsity calculation 4.0 times.
Number tokens in sparsity calculation window: 2.05e+06


In [28]:
activations_store = ActivationsStore(config, model)

Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

Dataset is tokenized! Updating config.


In [29]:
batch = activations_store.next_batch()

In [38]:
from sae_training.sparse_autoencoder import SparseAutoencoder

In [42]:
sae = SparseAutoencoder(config)
sae.eval()

SparseAutoencoder(
  (hook_sae_in): HookPoint()
  (hook_hidden_pre): HookPoint()
  (hook_hidden_post): HookPoint()
  (hook_sae_out): HookPoint()
)

In [45]:
sae_out, feature_acts, loss, mse_loss, l1_loss, mse_loss_ghost_resid = sae(batch)

In [60]:
import torch
import os
import sys

os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["WANDB__SERVICE_WAIT"] = "300"

dataset_path = 'taufeeque/othellogpt'
model_name = 'othello-gpt'
device = "cuda" if torch.cuda.is_available() else "cpu"

from sae_training.config import LanguageModelSAERunnerConfig
from sae_training.lm_runner import language_model_sae_runner

for exp_factor in [1, 2, 4, 8]:
    config = LanguageModelSAERunnerConfig(
        model_name=model_name,
        hook_point="blocks.6.hook_resid_pre",
        hook_point_layer=6,
        dataset_path=dataset_path,
        context_size=59,
        d_in=512,
        n_batches_in_buffer=32,
        total_training_tokens=50*(1e6), # prev: 10*(1e6)
        store_batch_size=32,
        device=device,
        seed=42,
        dtype=torch.float32, # todo: float16
        b_dec_init_method="mean", # todo: geometric_median
        expansion_factor=exp_factor, # todo: adjust
        l1_coefficient=0.0005, # prev: 0.001, 0.0001
        lr=0.00003, # prev: 0.0003
        lr_scheduler_name="constantwithwarmup",
        lr_warm_up_steps=5000,
        train_batch_size=4096,
        use_ghost_grads=True,
        feature_sampling_window=500,
        dead_feature_window=1e6,
        log_to_wandb=True,
        wandb_project="othello_gpt_sae",
        wandb_log_frequency=30,
        n_checkpoints=0,
        checkpoint_path="checkpoints"
    )

    sparse_autoencoder = language_model_sae_runner(config)

Run name: 4096-L1-0.0001-LR-3e-05-Tokens-5.000e+07
n_tokens_per_buffer (millions): 0.060416
Lower bound: n_contexts_per_buffer (millions): 0.001024
Total training steps: 12207.0
Total wandb updates: 406.0
n_tokens_per_feature_sampling_window (millions): 120.832
n_tokens_per_dead_feature_window (millions): 241664.0
Using Ghost Grads.
We will reset the sparsity calculation 24.0 times.
Number tokens in sparsity calculation window: 2.05e+06
Loaded pretrained model othello-gpt into HookedTransformer
Moving model to device:  cuda


Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

Dataset is tokenized! Updating config.


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


...

53| MSE Loss 0.104 | L1 0.136:   0%|          | 217088/50000000.0 [00:02<09:53, 83950.78it/s]/opt/conda/envs/mats_sae_training/lib/python3.11/site-packages/wandb/sdk/wandb_run.py:2171: UserWarning: Run (cyte7k9g) is finished. The call to `_console_raw_callback` will be ignored. Please make sure that you are using an active run.
  lambda data: self._console_raw_callback("stderr", data),
12207| MSE Loss 0.007 | L1 0.009: : 50003968it [10:42, 77857.91it/s]
12207| MSE Loss 0.006 | L1 0.009: 100%|█████████▉| 49999872/50000000.0 [10:15<00:00, 82640.15it/s]

Saved model to checkpoints/ptxlbu92/final_sparse_autoencoder_othello-gpt_blocks.6.hook_resid_pre_8192.pt


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


12207| MSE Loss 0.006 | L1 0.009: : 50003968it [10:27, 82640.15it/s]                              

details/current_learning_rate,▁▁▂▂▃▃▄▄▄▅▅▆▆▇▇█████████████████████████
details/n_training_tokens,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
losses/ghost_grad_loss,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
losses/l1_loss,██▇▆▅▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
losses/mse_loss,█▆▄▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
losses/overall_loss,██▆▅▄▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
metrics/CE_loss_score,▁▃▅▇▆▇▇▇▇▇▇█████████████████████████████
metrics/ce_loss_with_ablation,█▆▆▁▆▅▄▅▅▄▇▅▂▃▄▃▄▇▇▆▇▄▆▃▂▃▂▄▄▇█▃▄▅▅▅▄▆▅▇
metrics/ce_loss_with_sae,█▆▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
metrics/ce_loss_without_sae,▃▅▂▇▅▅▄▁▄▅▄▆▄▇▇▇▅▅▆▆▇▇▃▃▄▆▃▆▆█▇▄▇█▇▇▅█▆▇
metrics/explained_variance,▁▃▅▅▆▆▇▆▇▇▇▇▇▇▇▇████████████████████████
